# Time Series Visualization and Projection

Leveraging 20 years of Temperature and Drought Condition data at a county (~3100) level to illustrate relevant climate information

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings 

### Data Imports and Formatting

In [ ]:
mon = pd.read_csv('../../data/clean-data/Monthly_Temp_Drought_Combo.csv', dtype = {'FIPS':str})

In [ ]:
mon.drop(columns = 'Unnamed: 0', inplace = True)

In [ ]:
mon.head()

In [ ]:
mon.dtypes

In [ ]:
mon.rename(columns = {
    'Month' : 'month',
    'FIPS' : 'fips',
    'Tmin_C' : 'min_temp',
    'Tmax_C' : 'max_temp',
    'Tmean_C' : 'mean_temp',
    'Flag_T' : 'flag_pop_covered'
}, inplace = True)

In [ ]:
mon['month'] = pd.to_datetime(mon['month'],format = ('%Y-%m'))
mon['month'].head()

In [ ]:
mon['fips'] = mon['fips'].str.rjust(5,'0')
#cite :https://stackoverflow.com/a/339024 for rjust
mon['fips'].head()

In [ ]:
mon.head()

In [ ]:
# Convert Celsius to Farenheit to limit confusion within the U.S. Market
mon[['min_temp','max_temp','mean_temp']] *= (9/5)
mon[['min_temp','max_temp','mean_temp']] += 32

In [ ]:
# Confirm conversion of tepmerature columns
mon.head()

In [ ]:
mon.dtypes

#### Import Annual Summarized Data

In [ ]:
year = pd.read_csv('../../clean-data/Temp_Drought_Combo.csv', dtype= {'FIPS' : str})
year.drop(columns = 'Unnamed: 0', inplace = True)
year['year'] = pd.to_datetime(year['year'].astype(str))
year['FIPS'] = year['FIPS'].str.rjust(5,'0')
# year.set_index('year', inplace = True)

In [ ]:
year.head()

#### Create County Dictionary to enable Human Readable Outputs

In [ ]:
counties_df = pd.read_csv('../../data/raw-data/counties.csv', dtype = {'FIPS': str})
counties_df.drop(columns = 'Unnamed: 0', inplace = True)
counties_df['FIPS'] = counties_df['FIPS'].str.rjust(5,'0')

counties_df.head()

county_dict = dict(zip(counties_df['FIPS'], zip(counties_df['STATE'], counties_df['COUNTYNAME'], counties_df['LON'], counties_df['LAT'])))

## Local Time Series

In [ ]:
# FIPS Codes Filter
county_fips = '06001'
min_year = 2010
county = county_dict.get(county_fips)[1]
state = county_dict.get(county_fips)[0]

plot_temp_trends_county(county_fips, min_year, county, state)
plot_drought_trends_county(county_fips, min_year, county, state)

In [ ]:
def plot_temp_trends_county(county_fips, min_year, county, state):    

    # Filtered Monthly Summary View
    county_month_view_df = mon[(mon['month'].dt.year >= min_year) & (mon['fips'] == county_fips)]
    county_month_view_df.set_index('month', inplace = True)

    # Annual Summary from Daily Data
    county_year_view_df = year[(year['year'].dt.year >= min_year) & (year['FIPS'] == county_fips)]
    county_year_view_df = county_year_view_df[['year','FIPS','Tmean_C']]
    # Convert to Farenheit
    county_year_view_df['Tmean_C'] *= (9/5)
    county_year_view_df['Tmean_C'] += 32
    county_year_view_df.rename(columns = {'Tmean_C' : 'Tmean_F'}, inplace = True)

    county_year_view_df.set_index('year', inplace = True)
    #cite: Time Series in Pandas Lesson
    
    
    # Plot
    plt.figure(figsize = (12,8))
    plt.plot(county_month_view_df['min_temp'], c = '#EED78D', label = 'Low Temp (F)')
    plt.plot(county_month_view_df['max_temp'], c = '#C22B26',  label = 'High Temp (F)')
    plt.plot(county_month_view_df['mean_temp'], c = '#FFB632',  label = 'Mean Temp (F)')
    plt.plot(county_year_view_df['Tmean_F'], c = 'k', label = 'Annual Mean Temp (F)',)
    
    X1 = county_month_view_df['max_temp']
    X2 = county_month_view_df['min_temp']
    plt.fill_between(county_month_view_df.index, X1,X2, color = '#EED78D', alpha = 0.10)

    # Horizontal line at the min_year baseline
    plt.axhline(county_year_view_df.loc['2010', 'Tmean_F'][0], c = '#808080', label = f'{min_year} Mean Annual Temp Baseline (F)') 

    plt.title(f"Temperature Trend for {county} County, {state}")
    plt.yticks(fontsize = 12)
    plt.xticks(fontsize = 12)
    plt.ylabel('Average Monthly Temperature (F)', fontsize = 12)
    plt.legend()
    plt.savefig(fname = f'images/temp_trend_{county}_{state}.png', dpi = 200, bbox_inches = 'tight')
    plt.show()

In [ ]:
def plot_drought_trends_county(county_fips, min_year, county, state):
    
    # Filtered Monthly Summary View
    county_month_view_df = mon[(mon['month'].dt.year >= min_year) & (mon['fips'] == county_fips)]
    county_month_view_df.set_index('month', inplace = True)
    county_month_view_df['extreme_plus'] = county_month_view_df[['exceptional_drought','extreme_drought']].max(axis = 1)
    county_month_view_df['severe_plus'] = county_month_view_df[['exceptional_drought','extreme_drought','severe_drought']].max(axis = 1)
    county_month_view_df['moderate_plus'] = county_month_view_df[['exceptional_drought','extreme_drought','severe_drought', 'moderate_drought']].max(axis = 1)

    # Annual Summary from Daily Data
    county_year_view_df = year[(year['year'].dt.year >= min_year) & (year['FIPS'] == county_fips)]
    county_year_view_df = county_year_view_df[['year','FIPS','Tmean_C']]
    # Convert to Farenheit
    county_year_view_df['Tmean_C'] *= (9/5)
    county_year_view_df['Tmean_C'] += 32
    county_year_view_df.rename(columns = {'Tmean_C' : 'Tmean_F'}, inplace = True)

    county_year_view_df.set_index('year', inplace = True)
    #cite: Time Series in Pandas Lesson
    
    
    
    # Plot and Fill
    plt.figure(figsize = (12,8))

    plt.plot(county_month_view_df['moderate_plus'], c = '#EED78D',  label = 'Moderate Drought')
    plt.fill_between(county_month_view_df.index, county_month_view_df['moderate_plus'], 0, color = '#EED78D', alpha = 0.1)
    
    plt.plot(county_month_view_df['severe_plus'], c = '#FFB632',  label = 'Severe Drought')
    plt.fill_between(county_month_view_df.index, county_month_view_df['severe_plus'], 0, color = '#FFB632', alpha = 0.125)

    plt.plot(county_month_view_df['extreme_plus'], c = '#D58900',  label = 'Extreme Drought')
    plt.fill_between(county_month_view_df.index, county_month_view_df['extreme_plus'], 0, color = '#D58900', alpha = 0.15)
    
    plt.plot(county_month_view_df['exceptional_drought'], c = '#C22B26', label = 'Exceptional Drought')
    plt.fill_between(county_month_view_df.index, county_month_view_df['exceptional_drought'], 0, color = '#C22B26', alpha = 0.2)

    plt.title(f"Average Minimum Drought Condition for {county} County, {state}")
    plt.yticks(fontsize = 12)
    plt.xticks(fontsize = 12)
    plt.ylabel('Percent Population Experiencing Designated Drought Condition or Worse', fontsize = 12)
    plt.legend()
    plt.savefig(fname = f'images/drought_trend_{county}_{state}.png', dpi = 200, bbox_inches = 'tight')
    plt.show();

### Time Series Choropleths

In [ ]:
# cite : https://python-graph-gallery.com/choropleth-map-plotly-python/
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

#### County > State Time Choropleths

In [ ]:
county_fips = '06001'
min_year = 2010

# County Center latitude and longitude
county = county_dict.get(county_fips)[1]
state = county_dict.get(county_fips)[0]
county_lat = county_dict.get(county_fips)[3]
county_lon = county_dict.get(county_fips)[2]

# Filter down for State of Selected County
view_mon = mon[(mon['month'].dt.year >= min_year) & (mon['fips'].str[:2] == county_fips[:2])]
view_mon = pd.merge(left = view_mon, right = counties_df[['COUNTYNAME', 'STATE','FIPS']], left_on = 'fips', right_on = 'FIPS', how = 'left')
view_mon['month'] = view_mon['month'].astype(str)

# Filter down Geography json file used by the choropleth to relevant state to limit runtime
# Cite: for the idea on filtering down the geojson: https://stackoverflow.com/questions/70950535/plot-single-state-choropleth-map-in-plotly-how-to-index-geojson
counties_copy = counties.copy()
counties_copy['features'] = [feat for feat in counties['features'] if feat['properties']['STATE'] == county_fips[:2]]

# view_mon.head()
# Plot Choropleths
# cite: https://medium.com/p/2ff04921c60b

#Exceptional Drought
px.choropleth(data_frame = view_mon,
         lat=None,
         lon=None,
         locations='fips',
         geojson=counties_copy,
         animation_frame='month',
         color='exceptional_drought',labels = 'Exceptional Drought',
         color_continuous_scale='Reds',
         range_color=[0,100],
         hover_name='COUNTYNAME',
         hover_data=['COUNTYNAME','STATE','min_temp', 'max_temp', 'mean_temp','normal_wet', 'abnormally_dry', 'moderate_drought', 'severe_drought', 'extreme_drought', 'exceptional_drought'],
         scope = 'usa', center = {'lat' : county_lat, 'lon' : county_lon},
         title = f'Percent Population Experiencing Exceptional Drought: {state}',
         height=1000)

In [ ]:
#Extreme Drought or worse
view_mon['extreme_plus'] = view_mon[['exceptional_drought','extreme_drought']].max(axis = 1)
px.choropleth(data_frame = view_mon,
         lat=None,
         lon=None,
         locations='fips',
         geojson=counties_copy,
         animation_frame='month',
         color='extreme_plus',labels = 'Extreme Drought',
         color_continuous_scale='Oranges',
         range_color=[0,100],
         hover_name='COUNTYNAME',
         hover_data=['COUNTYNAME','STATE','min_temp', 'max_temp', 'mean_temp','normal_wet', 'abnormally_dry', 'moderate_drought', 'severe_drought', 'extreme_drought', 'exceptional_drought'],
         scope = 'usa', center = {'lat' : county_lat, 'lon' : county_lon},
         title = f'Percent Population Experiencing Extreme Drought: {state}',
         height=1000)



In [ ]:
#Severe Drought or worse
view_mon['severe_plus'] = view_mon[['exceptional_drought','extreme_drought','severe_drought']].max(axis = 1)
px.choropleth(data_frame = view_mon,
         lat=None,
         lon=None,
         locations='fips',
         geojson=counties_copy,
         animation_frame='month',
         color='severe_plus',labels = 'Severe Drought',
         color_continuous_scale='Oranges',
         range_color=[0,100],
         hover_name='COUNTYNAME',
         hover_data=['COUNTYNAME','STATE','min_temp', 'max_temp', 'mean_temp','normal_wet', 'abnormally_dry', 'moderate_drought', 'severe_drought', 'extreme_drought', 'exceptional_drought'],
         scope = 'usa', center = {'lat' : county_lat, 'lon' : county_lon},
         title = f'Percent Population Experiencing Severe Drought: {state}',
         height=1000)


In [ ]:
#Moderate Drought or worse
view_mon['moderate_plus'] = view_mon[['exceptional_drought','extreme_drought','severe_drought', 'moderate_drought']].max(axis = 1)
px.choropleth(data_frame = view_mon,
         lat=None,
         lon=None,
         locations='fips',
         geojson=counties_copy,
         animation_frame='month',
         color='moderate_plus',labels = 'Moderate Drought',
         color_continuous_scale='Oranges',
         range_color=[0,100],
         hover_name='COUNTYNAME',
         hover_data=['COUNTYNAME','STATE','min_temp', 'max_temp', 'mean_temp','normal_wet', 'abnormally_dry', 'moderate_drought', 'severe_drought', 'extreme_drought', 'exceptional_drought'],
         scope = 'usa', center = {'lat' : county_lat, 'lon' : county_lon},
         title = f'Percent Population Experiencing Moderate Drought: {state}',
         height=1000)

#### Temperature Change by County

In [ ]:
# Format Temperature Data with at 12 month lag
view_mon_c = view_mon.copy()
view_mon_c['month'] = pd.to_datetime(view_mon_c['month'])
view_mon_c.set_index('month', inplace = True)
view_mon_c['min_12'] = view_mon_c['min_temp'].diff(12)
view_mon_c['max_12'] = view_mon_c['max_temp'].diff(12)
view_mon_c['mean_12'] = view_mon_c['mean_temp'].diff(12)
view_mon_12 = view_mon_c.dropna()
view_mon_12 = view_mon_12[['fips','min_temp','max_temp','mean_temp','COUNTYNAME','STATE','min_12','max_12','mean_12']]
view_mon_12.reset_index(inplace = True)
view_mon_12['month'] =  view_mon_12['month'].astype(str)
# view_mon_12.head()



px.choropleth(data_frame = view_mon_12,
         lat=None,
         lon=None,
         locations='fips',
         geojson=counties_copy,
         animation_frame='month',
         color='mean_12',labels = '12 Month Mean Temperature Change',
         color_continuous_scale='rdbu_r',
         range_color=[-40,40],
         hover_name='COUNTYNAME',
         hover_data=['COUNTYNAME','STATE','min_temp', 'max_temp', 'mean_temp'],
         scope = 'usa', 
           center = {'lat' : county_lat, 'lon' : county_lon},
         title = f'Year over Year Monthly Average Temperature Change (F): {state}',
         height=1000)

### Seasonality Decomposition

In [ ]:
from statsmodels.tsa.seasonal import STL, seasonal_decompose
# cite: Time Series Lesson with Matt Brems

# Data Prep
view_mon_mean_temp = view_mon[['month','mean_temp','min_temp','max_temp']][view_mon['fips'] == county_fips]
view_mon_mean_temp['month'] = pd.to_datetime(view_mon_mean_temp['month'])
view_mon_mean_temp.set_index('month', inplace = True)

In [ ]:
decomp_mean = STL(view_mon_mean_temp['mean_temp']).fit()

fig, ax = plt.subplots(2,1, figsize = (8,8))
fig.tight_layout(h_pad=3)
fig.subplots_adjust(top = 0.92)
plt.suptitle(f"Mean Monthly Temperature STL Decomposition: {county} County, {state}", fontsize = 13)
# ax[0].sup_title()

ax[0].set_title(f'STL Seasonally Adjusted Monthly Mean Temperature Trend',fontsize = 11)
ax[0].set_ylabel('Seasonally Adjusted Temperature (F)')
decomp_mean.trend.plot(ax=ax[0], c='#FFB632')

ax[1].set_title(f'Monthly Average Temperature - Seasonality Impact')
ax[1].set_ylabel('Seasonality Impact: Monthly Average Temperature (F)')
decomp_mean.seasonal.plot(ax=ax[1], c = '#FFB632')

# ax[2].set_title(f'Monthly Average Temperature - Residuals')
# ax[2].set_ylabel('Residual Temperature Variation (F)')
# decomp_mean.resid.plot(ax=ax[2], c= '#FFB632');
;
plt.savefig(dpi = 200, fname = f'images/MonthlyAverageTemperature_Decomp_{county}_{state}.png', bbox_inches = 'tight')

In [ ]:
decomp_max = STL(view_mon_mean_temp['max_temp']).fit()

fig, ax = plt.subplots(2,1, figsize = (8,8))
fig.tight_layout(h_pad=3)
fig.subplots_adjust(top = 0.92)
plt.suptitle(f"Monthly Average High Temperature STL Decomposition: {county} County, {state}", fontsize = 13)

ax[0].set_title(f'STL Seasonally Adjusted Monthly High Temperature Trend',fontsize = 11)
ax[0].set_ylabel('Seasonally Adjusted Temperature (F)')
decomp_max.trend.plot(ax=ax[0], c='#C22B26')

ax[1].set_title(f'Monthly Average High Temperature - Seasonality Impact')
ax[1].set_ylabel('Seasonality Impact: Monthly Average Temperature (F)')
decomp_max.seasonal.plot(ax=ax[1], c = '#C22B26')

# ax[2].set_title(f'Monthly Average High Temperature - Residuals')
# ax[2].set_ylabel('Residual Temperature Variation (F)')
# decomp_mean.resid.plot(ax=ax[2], c= '#C22B26');
;
plt.savefig(dpi = 200, fname = f'images/MonthlyHighTemperature_Decomp_{county}_{state}.png', bbox_inches = 'tight')